# EDA

In this EDA, the study dataset are examined in more detail. The goal is to understand the data and the structure of it better.


In [87]:
import sys
import os
sys.path.append(os.path.abspath('../')) ## needed to import the function.py file

from functions import *
import pandas as pd
import plotly.express as px
import plotly.io as pio
import spacy
import xlsxwriter

# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('stopwords')
#!python -m spacy download de_core_news_lg
## download nlp language package

# Load the German model
nlp = spacy.load("de_core_news_lg")

### Load Data

In [88]:
## Import dataframe for overview with all Surveys from 2019-2022
filelocation = '../../data/DataClean'
df = pd.read_feather(filelocation)

## Import textbased dataframe with all Surveys from 2019-2022 with text comments and prerocessed columns
filelocation = '../../data/DataText'
df_text = pd.read_feather(filelocation)

## load config file
config = pd.read_excel('../../config/config.xlsx',sheet_name='fragecodes')
invites_month = pd.read_excel('../../config/config.xlsx',sheet_name='invites')

In [89]:
#df.info()
#df_text.info()


## EDA

Possiblity to filter on a specific timerange. The filter is applied the whole EDA.

In [90]:
# startdate = '2019-01-01'
# enddate = '2022-12-31'

# # # Apply time selection for all DataFrames
# df= df[(df['u_date'] > startdate) & (df['u_date'] <= enddate)]

### Overview

#### General DataFrame Overview

In [91]:
df = df.sort_values("u_date",ascending=False)
df = add_date_columns(df, "u_date")

In [92]:
#df.tail(5)
df.head(5)

,participant_id,u_date,year,month,quarter,yearmonth,yearquarter,season,yearseason,Kommentar,...,ft_vm_kurz,ft_zielort_uic,fg_abfahrt,fg_ankunft,fg_startort_uic,fg_zielort_uic,fg_startort,fg_zielort,ft_startort,ft_zielort
239965,604504,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Ich bin Pendlerin ( und glückliche Inhaberin e...,...,IR,8509000,11:15:00,15:24:00,8509251,8500207,Samedan,Solothurn,Samedan,Chur
239976,604518,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Immer wieder grosses Problem ist kaputt gescho...,...,NaN,NaN,None,None,NaN,NaN,BÜLACH,ZÜRICH HB,BÜLACH,ZÜRICH HB
239979,604523,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,<NA>,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
239954,604490,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,<NA>,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
239988,604534,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,eventuell ein sharing-Angebot mobility-sbb vom...,...,IC,8505000,13:21:00,15:41:00,8505493,8505000,Cavigliano,Luzern,Bellinzona,Luzern


In [93]:
df.shape

(240053, 48)

### TextData

In [94]:
# usage example:
# df_text = add_date_columns(df_text, 'u_date')
# df_text = df.dropna(subset=["Kommentar"])
# df_text = df_text[df_text.Kommentar.apply(lambda x: len(str(x))>=3)] # min 3 characters for valid comment

In [95]:
df_text

,participant_id,u_date,year,month,quarter,yearmonth,yearquarter,season,yearseason,Kommentar,...,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR,text_preprocessed,text_preprocessed_tokenized,lemmatized,nouns,adjectives,verbs,nouns_adjectives_and_verbs
0,612374,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Häufigere Verbindungen zw. Bern-Luzern.,...,4,4,100.000000,häufigere verbindungen zw luzern,"[häufigere, verbindungen]",None,[verbindungen],[häufigere],[],"[häufigere, verbindungen]"
1,604479,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,"Als Student ist das GA Preiswert, aber mehr wü...",...,32,28,87.500000,student ga preiswert gerne bezahlen monatslohn...,"[student, ga, preiswert, gerne, bezahlen, mona...",None,"[student, monatslohn, preis]",[],[bezahlen],"[student, bezahlen, monatslohn, preis]"
2,604528,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Weniger 1. Klasswagons und mehr 2. Klassenwago...,...,23,22,95.652174,klasswagons klassenwagons genau bern bern zt ...,"[klasswagons, klassenwagons, genau, , zt, kla...",None,"[klasswagons, abteil]",[],[],"[klasswagons, abteil]"
3,604518,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Immer wieder grosses Problem ist kaputt gescho...,...,12,12,100.000000,problem kaputt geschossen wc züge,"[problem, kaputt, geschossen, wc, züge]",None,"[problem, züge]",[],[geschossen],"[problem, geschossen, züge]"
4,604504,2022-12-31,2022,12,4,2022-12-01,2022Q4,winter,2022-winter,Ich bin Pendlerin ( und glückliche Inhaberin e...,...,52,46,88.461538,pendlerin glückliche inhaberin gas pendle fas...,"[pendlerin, , glückliche, inhaberin, gas, pen...",None,"[pendlerin, inhaberin, züge, klasse]",[glückliche],"[pendle, verkürzt, geführt, stehen]","[pendlerin, glückliche, inhaberin, pendle, züg..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64340,51540,2019-01-01,2019,1,1,2019-01-01,2019Q1,winter,2019-winter,Dieser Zug war ab Zürich HB ein Nachtzug. Die ...,...,34,32,94.117647,zug nachtzug app nachtzuschlag verrechnet pass...,"[zug, nachtzug, app, nachtzuschlag, verrechnet...",None,"[zug, nachtzug, kontrolle]",[],"[verrechnet, passiert, gekommen, lösen]","[zug, nachtzug, verrechnet, passiert, kontroll..."
64341,51531,2019-01-01,2019,1,1,2019-01-01,2019Q1,winter,2019-winter,Übersichtlichkeit der App.Es könnte besser ang...,...,23,23,100.000000,übersichtlichkeit appes angegeben werden ticke...,"[übersichtlichkeit, appes, angegeben, ticket, ...",None,"[appes, ticket, korrekturfunktion, kauf, tickets]","[vorgegebener, versehentlichen, falschen]","[angegeben, gekauft]","[appes, angegeben, ticket, gekauft, korrekturf..."
64342,51529,2019-01-01,2019,1,1,2019-01-01,2019Q1,winter,2019-winter,Bei der Rückfahrt hatten wir Sitzplätze aber b...,...,73,60,82.191781,rückfahrt sitzplätze anreise stehen schlimm rü...,"[rückfahrt, sitzplätze, anreise, stehen, schli...",None,"[rückfahrt, sitzplätze, anreise, sitzplätze, k...",[],"[stehen, gewundert, belegt, sitzen, verstaut]","[rückfahrt, sitzplätze, anreise, stehen, gewun..."
64343,51527,2019-01-01,2019,1,1,2019-01-01,2019Q1,winter,2019-winter,Mehr Spielwagen auf unterschiedlichen Strecken...,...,7,7,100.000000,spielwagen unterschiedlichen strecken super,"[spielwagen, unterschiedlichen, strecken, super]",None,"[spielwagen, strecken]",[unterschiedlichen],[],"[spielwagen, unterschiedlichen, strecken]"


#### Number of comments per month

In [96]:
df_text_counts = pd.DataFrame(df_text.groupby(['yearmonth'])['participant_id'].count()).reset_index()
df_participantstotals_counts = pd.DataFrame(df.groupby(['yearmonth'])['participant_id'].count()).reset_index()

In [97]:
# Merge Dataframes
df_counts_month = df_text_counts.merge(df_participantstotals_counts, on='yearmonth', how='left')
df_counts_month = df_counts_month.merge(invites_month, on='yearmonth', how='left') # join aount of invites
df_counts_month.rename(columns = {'participant_id_x':'Anzahl Befragte mit Kommentar','participant_id_y':'Anzahl Befragte'}, inplace = True)
df_counts_month['Kommentar Ratio in %'] = (df_counts_month['Anzahl Befragte mit Kommentar']/df_counts_month['Anzahl Befragte'])*100
df_counts_month['Rücklauf Ratio in %'] = (df_counts_month['Anzahl Befragte']/df_counts_month['Anzahl Einladungen'])*100

In [98]:
df_counts_month.describe()

,Anzahl Befragte mit Kommentar,Anzahl Befragte,Anzahl Einladungen,Kommentar Ratio in %,Rücklauf Ratio in %
count,46.000000,46.000000,46.000000,46.000000,46.000000
mean,1398.804348,5218.543478,77079.195652,26.622868,6.724160
std,501.099264,1734.978600,21349.529601,2.501516,0.884570
min,631.000000,2314.000000,42605.000000,22.389111,4.752712
25%,997.250000,4044.000000,60562.500000,24.612121,6.208061
50%,1322.500000,5111.500000,73199.000000,25.914002,6.726789
75%,1738.750000,5960.500000,89753.750000,28.269197,7.140620
max,2684.000000,9898.000000,122993.000000,31.763001,9.891194


In [99]:
len(df_text)

64345

In [100]:
## plot settings
color_discrete_sequence = color_discrete_sequence #imported from globalvars.py
template = template ##imported from globalvars.py

In [101]:
df_counts_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   yearmonth                      46 non-null     datetime64[ns]
 1   Anzahl Befragte mit Kommentar  46 non-null     int64         
 2   Anzahl Befragte                46 non-null     int64         
 3   Anzahl Einladungen             46 non-null     float64       
 4   Kommentar Ratio in %           46 non-null     float64       
 5   Rücklauf Ratio in %            46 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 2.5 KB


In [102]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y=['Anzahl Befragte mit Kommentar', 'Anzahl Befragte','Anzahl Einladungen'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)

fig.update_layout(
    title="Anzahl Einladungen, Anzahl Befragungen & Anzahl Befragte mit Kommentar",
    yaxis_title='',
    xaxis_title='',
    width=900, 
    height=450,
    legend_title="")

fig.update_traces(line_width=2)
fig.update_traces(patch={"line": {"dash": 'dot'}}, selector={"legendgroup": "Anzahl Einladungen"})

fig.update_yaxes(type="log") 

fig.add_vrect(
    x0="2020-03-01",
    x1="2020-05-31",
    line_width=0,
    fillcolor="#0B1F26",
    opacity=0.2,
    annotation_text="Pause",
    annotation_position="top left",
    annotation_font_size=10,
    annotation_font_color="#0B1F26",
)

fig.update_traces(connectgaps=False)

fig.show()

pio.write_image(fig,"../../exports/images/fig-anteil_befragte_und_anzahl_kommentare.svg") #save fig as svg

In [103]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y='Kommentar Ratio in %',
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Anteil Befragte mit Kommentar",
    yaxis_title='Anteil in %',
    xaxis_title='Monat',
    width=900, 
    height=450,
    )
fig.update_traces(line_width=2)
#fig.update_yaxes(range=(0,40))


# show legend for singel line plots
fig['data'][0]['showlegend'] = True
fig['data'][0]['name'] = 'Anteil Kommentare in %'

fig.update_yaxes(range=(21,35))

fig.show()

pio.write_image(fig,"../../exports/images/fig-anteil-befragte-mit-kommentar.svg") #save fig as svg

In [104]:
df_counts_month.describe()

,Anzahl Befragte mit Kommentar,Anzahl Befragte,Anzahl Einladungen,Kommentar Ratio in %,Rücklauf Ratio in %
count,46.000000,46.000000,46.000000,46.000000,46.000000
mean,1398.804348,5218.543478,77079.195652,26.622868,6.724160
std,501.099264,1734.978600,21349.529601,2.501516,0.884570
min,631.000000,2314.000000,42605.000000,22.389111,4.752712
25%,997.250000,4044.000000,60562.500000,24.612121,6.208061
50%,1322.500000,5111.500000,73199.000000,25.914002,6.726789
75%,1738.750000,5960.500000,89753.750000,28.269197,7.140620
max,2684.000000,9898.000000,122993.000000,31.763001,9.891194


#### Minimal, maximal and average length of Kommentar

In [105]:
df_text_average_character_median_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Character'].median()).reset_index()
df_text_average_character_mean_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Character'].mean()).reset_index()

In [106]:
# Merge Dataframes
df_text_character_counts_month = df_text_average_character_median_per_month.merge(df_text_average_character_mean_per_month, on='yearmonth', how='left')
df_text_character_counts_month.rename(columns = {'Kommentar_Character_x':'Median Characters','Kommentar_Character_y':'Mittelwert Characters'}, inplace = True)

In [107]:
fig = px.line(df_text_character_counts_month,
              x="yearmonth",
              y=['Median Characters','Mittelwert Characters'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Freitextkommentare: Characters-Median & Characters-Mittelwert in Vergleich",
    yaxis_title='Anzahl',
    xaxis_title='Monat',
    width=900, 
    height=450,
    legend_title="")
fig.update_traces(line_width=2)
fig.update_yaxes(range=(81,200))



fig.show()

pio.write_image(fig,"../../exports/images/fig-Characters-median-mittelwert.svg") #save fig as svg

In [108]:
fig = px.histogram(
    df_text,
    x="Kommentar_Character",
    range_x = [1,1500],
    title="Freitextkommentare: Häufigkeitsverteilung Länge der Kommentare (Characters)",
    color = "year",
    labels={'Kommentar_Character':'Anzahl Character'},
    color_discrete_sequence=color_discrete_sequence,
    width=900, 
    height=450,
    template=template)
    
fig.update_layout(yaxis_title="Anzahl Kommentare")

# Overlay both histograms
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)

fig.show()

pio.write_image(fig,"../../exports/images/fig-character-hist.svg") #save fig as svg

In [109]:
df_text.describe()

,participant_id,year,month,quarter,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,wime_fahrplan,wime_oes_fahrt,S_alter,u_preis,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR
count,64345.000000,64345.000000,64345.000000,64345.000000,23999.000000,51596.000000,52303.000000,52482.000000,52789.000000,54788.000000,60378.000000,63477.000000,50105.000000,64192.000000,56179.000000,64345.000000,64345.000000,64345.000000,64345.000000
mean,372935.377823,2020.441324,6.788872,2.614640,88.293494,74.376944,76.517228,86.943725,75.383603,78.793055,67.834771,78.402545,89.456752,52.124315,43.184389,173.492641,26.906162,23.996752,94.703922
std,179482.611570,1.208423,3.395010,1.109919,19.803944,25.620081,23.203525,24.416253,30.260222,23.290824,29.190136,27.688076,16.154632,15.624052,269.744002,181.764702,28.570797,22.535126,6.900195
min,41305.000000,2019.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,1.000000,1.000000,33.333333
25%,123881.000000,2019.000000,4.000000,2.000000,77.777778,55.555556,66.666667,77.777778,55.555556,75.000000,50.000000,66.666667,77.777778,41.000000,8.800000,59.000000,9.000000,9.000000,90.740741
50%,426230.000000,2020.000000,7.000000,3.000000,100.000000,75.000000,77.777778,100.000000,88.888889,77.777778,75.000000,88.888889,100.000000,54.000000,17.400000,119.000000,18.000000,17.000000,100.000000
75%,516461.000000,2022.000000,10.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,64.000000,31.300000,218.000000,34.000000,32.000000,100.000000
max,612374.000000,2022.000000,12.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,98.000000,6300.000000,2047.000000,347.000000,230.000000,100.000000


In [110]:
df_text.Kommentar_Tokens.quantile(0.7) 

30.0

In [111]:
#df_text[df_text.year == 2019][df_text.Kommentar_Character == 199]
#df_text

- Median of 119 Characters per Comment (140 original Twitter length)
- Median of 18 Tokens per comment.
- Average TTR of 94.7%

In [112]:
satisfaction_codes = config[config['vartype'] == 'satisfaction']['fragecode'] #loading satisfaction codes from config
df_text['rating_selected_average'] = df_text[satisfaction_codes].mean(axis=1) #calculate mean ratings 

satisfaction_category = pd.cut(df_text['rating_selected_average'],bins=[0,25,50,75,100],labels=["<25 Punkte","25-50 Punkte","50-75 Punkte","75-100 Punkte"])
df_text.insert(49,"rating_selected_average_group",satisfaction_category)

satisfaction_category_spontan = pd.cut(df_text['wime_gesamtzuf'],bins=[0,25,50,75,100],labels=["<25 Punkte","25-50 Punkte","50-75 Punkte","75-100 Punkte"])
df_text.insert(50,"rating_wime_gesamtzuf_group",satisfaction_category_spontan)

In [113]:
# x and y given as array_like objects
fig = px.scatter(
    df_text,
    x='Kommentar_Tokens', 
    y='Kommentar_TTR',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence,
    color = "rating_selected_average_group",
    marginal_x='box',
    category_orders={"rating_selected_average_group": ["75-100 Punkte","50-75 Punkte","25-50 Punkte","<25 Punkte"]}
    #color_continuous_scale=[(0,'red'), (0.75, 'orange'), (1,'green')]
                 )

fig.update_layout(
    title="Kommentarlänge und Type-Token Ratio im Vergleich mit Zufriedenheitsbewertung",
    yaxis_title='TTR in %',
    xaxis_title='Kommentarlänge in Tokens',
    width=900, 
    height=450,
    legend_title="")

#fig.update_yaxes(range=(55,105))

fig.show()

pio.write_image(fig,"../../exports/images/fig-Characters-TTR_scatter.svg") #save fig as svg

In [114]:
# # Pandas Profiling TextData
# profile = ProfileReport(
#     df_text[
#         [
#             #"participant_id",
#             #"u_date",
#             "Kommentar",
#             "Kommentar_Character",
#             "Kommentar_TTR",
#             "Kommentar_Tokens",
#             "Kommentar_Types",
#             'rating_selected_average',
#         ]
#     ],
#     title="TextData",
#     lazy=False,
#     dark_mode=True,
# )

# profile.to_file("html/ProfilingTextData.html")


### N-Grams

In [115]:
# create sub-dataframes for year specific plots
# df_text_2019 = df_text[df_text['year']==2019] # FIlter on specific year (optional)
# df_text_2020 = df_text[df_text['year']==2020] # FIlter on specific year (optional)
# df_text_2021 = df_text[df_text['year']==2021] # FIlter on specific year (optional)
# df_text_2022 = df_text[df_text['year']==2022] # FIlter on specific year (optional)

df_text = df_text
label_plots = "Gesamter-Zeitraum"
df_text_len = len(df_text )

N-grams are used to describe the number of words used as observation points, e.g., unigram means singly-worded, bigram means 2-worded phrase, and trigram means 3-worded phrase. In order to do this, we use scikit-learn’s CountVectorizer function.

#### Unigrams

##### Nouns

In [116]:
common_words = get_top_n_ngrams(corpus=join_list_of_list(df_text['nouns']), n=40, ngram_range=(1,1))
#common_words = get_top_n_ngrams(corpus=df_text['text_preprocessed'], n=10, ngram_range=(1,1))

unigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams['count_relative']=(unigrams['count']/df_text_len)
unigrams

,word,count,count_relative
0,zug,16992,0.264076
1,züge,4603,0.071536
2,klasse,4378,0.068039
3,fahrt,3601,0.055964
4,strecke,3198,0.049701
5,verbindung,2830,0.043982
6,verspätung,2642,0.041060
7,zügen,2399,0.037283
8,minuten,2250,0.034968
9,leute,2138,0.033227


In [117]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(unigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Unigrams im gesamten Betrachtungszeitraum",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Unigrams-Substantive-"+label_plots+"-barchart.svg") #save fig as svg

##### Verbs

In [118]:
common_words = get_top_n_ngrams(corpus=join_list_of_list(df_text['verbs']), n=40, ngram_range=(1,1))
#common_words = get_top_n_ngrams(corpus=df_text['text_preprocessed'], n=10, ngram_range=(1,1))

unigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams['count_relative']=(unigrams['count']/df_text_len)
unigrams

,word,count,count_relative
0,fahren,2025,0.031471
1,umsteigen,1751,0.027213
2,lösen,1457,0.022644
3,fahre,1404,0.021820
4,stehen,1335,0.020748
5,wagen,1317,0.020468
6,nehmen,1133,0.017608
7,warten,1109,0.017235
8,fährt,1082,0.016816
9,sitzen,977,0.015184


In [119]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(unigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Unigrams im gesamten Betrachtungszeitraum",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Unigrams-Verbs-"+label_plots+"-barchart.svg") #save fig as svg

##### Adjectives

In [120]:
common_words = get_top_n_ngrams(corpus=join_list_of_list(df_text['adjectives']), n=40, ngram_range=(1,1))
#common_words = get_top_n_ngrams(corpus=df_text['text_preprocessed'], n=10, ngram_range=(1,1))

unigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams['count_relative']=(unigrams['count']/df_text_len)
unigrams

,word,count,count_relative
0,bessere,1478,0.022970
1,stosszeiten,905,0.014065
2,letzten,710,0.011034
3,verspätungen,684,0.010630
4,nächsten,563,0.008750
5,direkte,489,0.007600
6,mobile,487,0.007569
7,ältere,475,0.007382
8,günstigere,468,0.007273
9,freundliche,458,0.007118


In [121]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(unigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Unigrams im gesamten Betrachtungszeitraum",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Unigrams-Adjectives-"+label_plots+"-barchart.svg") #save fig as svg

##### Nouns, Verbs and Ajectives

In [122]:
common_words = get_top_n_ngrams(corpus=join_list_of_list(df_text['nouns_adjectives_and_verbs']), n=40, ngram_range=(1,1))
#common_words = get_top_n_ngrams(corpus=df_text['text_preprocessed'], n=10, ngram_range=(1,1))

unigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams['count_relative']=(unigrams['count']/df_text_len)
unigrams

,word,count,count_relative
0,zug,16992,0.264076
1,züge,4603,0.071536
2,klasse,4413,0.068583
3,fahrt,3602,0.055979
4,strecke,3238,0.050322
5,wagen,3028,0.047059
6,verbindung,2831,0.043997
7,zügen,2746,0.042676
8,verspätung,2642,0.041060
9,minuten,2250,0.034968


In [123]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(unigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Unigrams im gesamten Betrachtungszeitraum",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Unigrams-NVA_Combined-"+label_plots+"-barchart.svg") #save fig as svg

#### Bigrams

##### Nouns

In [124]:
common_words_bigrams = get_top_n_ngrams(corpus=join_list_of_list(df_text['nouns']), n=40, ngram_range=(2,2))

bigrams = pd.DataFrame(common_words_bigrams, columns = ['word' , 'count'])
bigrams['count_relative']=(bigrams['count']/df_text_len)
bigrams

,word,count,count_relative
0,zug zug,991,0.015401
1,zug verspätung,614,0.009542
2,klasse klasse,451,0.007009
3,minuten verspätung,409,0.006356
4,zug minuten,390,0.006061
5,verspätung zug,240,0.003730
6,fahrt zug,228,0.003543
7,leute zug,222,0.003450
8,zug klasse,208,0.003233
9,anschluss zug,193,0.002999


In [125]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(bigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Bigrams im gesamten Betrachtungszeitraum (Substantive)",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Bigrams-"+label_plots+"-barchart.svg") #save fig as svg



##### Verbs

In [126]:
common_words_bigrams = get_top_n_ngrams(corpus=join_list_of_list(df_text['verbs']), n=40, ngram_range=(2,2))

bigrams = pd.DataFrame(common_words_bigrams, columns = ['word' , 'count'])
bigrams['count_relative']=(bigrams['count']/df_text_len)
bigrams

,word,count,count_relative
0,lässt wünschen,186,0.002891
1,lösen lösen,87,0.001352
2,verkürzt geführt,58,0.000901
3,verpasst warten,51,0.000793
4,umsteigen umsteigen,45,0.000699
5,fahren fahren,45,0.000699
6,wagen wagen,43,0.000668
7,stehen sitzen,35,0.000544
8,fährt fährt,35,0.000544
9,umsteigen warten,33,0.000513


In [127]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(bigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Bigrams im gesamten Betrachtungszeitraum (Substantive)",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Bigrams-Verbs-"+label_plots+"-barchart.svg") #save fig as svg

#### Adjectives

In [128]:
common_words_bigrams = get_top_n_ngrams(corpus=join_list_of_list(df_text['adjectives']), n=40, ngram_range=(2,2))

bigrams = pd.DataFrame(common_words_bigrams, columns = ['word' , 'count'])
bigrams['count_relative']=(bigrams['count']/df_text_len)
bigrams

,word,count,count_relative
0,bessere bessere,32,0.000497
1,stosszeiten längere,26,0.000404
2,letzter verspätungen,24,0.000373
3,halbtax halbtax,21,0.000326
4,bessere schnellere,20,0.000311
5,ältere gehbehinderte,18,0.000280
6,letzten verspätungen,15,0.000233
7,halbe nächsten,13,0.000202
8,verspätungen zugsausfälle,11,0.000171
9,mobile mobile,11,0.000171


##### Nouns, Verbs and Ajectives

In [129]:
common_words_bigrams = get_top_n_ngrams(corpus=join_list_of_list(df_text['nouns_adjectives_and_verbs']), n=40, ngram_range=(2,2))

bigrams = pd.DataFrame(common_words_bigrams, columns = ['word' , 'count'])
bigrams['count_relative']=(bigrams['count']/df_text_len)
bigrams

,word,count,count_relative
0,zug verspätung,465,0.007227
1,minuten verspätung,399,0.006201
2,zug zug,334,0.005191
3,zug fährt,308,0.004787
4,zug fahren,305,0.004740
5,zug nehmen,293,0.004554
6,zug minuten,278,0.004320
7,fahre zug,274,0.004258
8,halbe stunde,266,0.004134
9,verfügung stellen,263,0.004087


In [130]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(bigrams.sort_values('count_relative', ascending=False),
    y='count_relative', 
    x='word',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Top 40 - Bigrams im gesamten Betrachtungszeitraum (Substantive, Verben und Adjektive)",
    xaxis_title='',
    yaxis_title='Häufigkeit',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-Bigrams-"+label_plots+"-barchart.svg") #save fig as svg

#### TF-IDF

#### TF-IDF nach Jahr

In [131]:
year_keywords_general = find_trending_keywords(df_text, filter_column='year', text_column='text_preprocessed_tokenized')
year_keywords_general = trending_keywords_to_dataframe(year_keywords_general)
#year_keywords_general
create_export_table(convert_resulttable_to_wide_format(year_keywords_general), filename='../../exports/tables/year_keywords_general.html')


In [132]:
year_keywords= find_trending_keywords_diff_normaized(df_text, filter_column='year', text_column='text_preprocessed_tokenized')
year_keywords = trending_keywords_to_dataframe(year_keywords)
#year_keywords
create_export_table(convert_resulttable_to_wide_format(year_keywords), filename=None)

#### TF-IDF nach Jahr und Quartal

In [133]:
yearquarter_keywords_general = find_trending_keywords(df_text, filter_column='yearquarter', text_column='text_preprocessed_tokenized')
yearquarter_keywords_general = trending_keywords_to_dataframe(yearquarter_keywords_general)
#yearquarter_keywords_general
create_export_table(convert_resulttable_to_wide_format(yearquarter_keywords_general), filename="../../exports/tables/yearquarter_keywords_general.html")

In [134]:
yearquarter_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='yearquarter', text_column='text_preprocessed_tokenized')
yearquarter_keywords = trending_keywords_to_dataframe(yearquarter_keywords)
#yearquarter_keywords_general
create_export_table(convert_resulttable_to_wide_format(yearquarter_keywords), filename="../../exports/tables/yearquarter_keywords_group_specific.html")

#### TF-IDF nach Jahr und Season

In [135]:
yearseason_keywords_general = find_trending_keywords(df_text, filter_column='yearseason', text_column='text_preprocessed_tokenized')
yearseason_keywords_general = trending_keywords_to_dataframe(yearseason_keywords_general)
# yearseason_keywords_general
create_export_table(convert_resulttable_to_wide_format(yearseason_keywords_general), filename="../../exports/tables/yearseason_keywords_general.html")

In [136]:
yearseason_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='yearseason', text_column='text_preprocessed_tokenized')
yearseason_keywords = trending_keywords_to_dataframe(yearseason_keywords)
# yearseason_keywords
create_export_table(convert_resulttable_to_wide_format(yearseason_keywords), filename="../../exports/tables/yearseason_keywords_group_specific.html")

#### TF-IDF nach Jahr und Monat

In [137]:
yearmonth_keywords_general = find_trending_keywords(df_text, filter_column='yearmonth', text_column='text_preprocessed_tokenized')
yearmonth_keywords_general = trending_keywords_to_dataframe(yearmonth_keywords_general)
#yearmonth_keywords
create_export_table(convert_resulttable_to_wide_format(yearmonth_keywords_general), filename="../../exports/tables/yearmonth_keywords_general.html")

In [138]:
yearmonth_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='yearmonth', text_column='text_preprocessed_tokenized')
yearmonth_keywords = trending_keywords_to_dataframe(yearmonth_keywords)
yearmonth_keywords
create_export_table(convert_resulttable_to_wide_format(yearmonth_keywords), filename="../../exports/tables/yearmonth_keywords_group_specific.html")


#### TF-IDF nach Saison

In [139]:
season_keywords_general = find_trending_keywords(df_text, filter_column='season', text_column='text_preprocessed_tokenized')
season_keywords_general = trending_keywords_to_dataframe(season_keywords_general)
#season_keyword
create_export_table(convert_resulttable_to_wide_format(season_keywords_general), filename="../../exports/tables/season_keywords_general.html")

In [140]:
season_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='season', text_column='text_preprocessed_tokenized')
season_keywords = trending_keywords_to_dataframe(season_keywords)
#season_keyword
create_export_table(convert_resulttable_to_wide_format(season_keywords), filename='../../exports/tables/season_keywords_group_specific.html')

#### TF-IDF nach Zielort

In [141]:
# Filter the dataframe by category count
counts = df_text.groupby('ft_zielort').size()
categories_to_keep = counts[counts >= 500].index #limit to bigger destinations
mask = df_text['ft_zielort'].isin(categories_to_keep)
filtered_df_orte = df_text[mask]

In [142]:
ft_zielort_keywords_general = find_trending_keywords(filtered_df_orte, filter_column='ft_zielort', text_column='text_preprocessed_tokenized')
ft_zielort_keywords_general = trending_keywords_to_dataframe(ft_zielort_keywords_general)
#ft_zielort_keywords
create_export_table(convert_resulttable_to_wide_format(ft_zielort_keywords_general), filename='../../exports/tables/ft_zielort_keywords_general.html')

In [143]:
ft_zielort_keywords = find_trending_keywords_diff_normaized(filtered_df_orte, filter_column='ft_zielort', text_column='text_preprocessed_tokenized')
ft_zielort_keywords = trending_keywords_to_dataframe(ft_zielort_keywords)
#ft_zielort_keywords
create_export_table(convert_resulttable_to_wide_format(ft_zielort_keywords), filename='../../exports/tables/ft_zielort_keywords_specific.html')

#### TF-TDF nach Reisezweck

In [144]:
reisezweck_keywords_general = find_trending_keywords(df_text, filter_column='R_zweck', text_column='text_preprocessed_tokenized')
reisezweck_keywords_general = trending_keywords_to_dataframe(reisezweck_keywords_general)
#reisezweck_keywords
create_export_table(convert_resulttable_to_wide_format(reisezweck_keywords_general), filename='../../exports/tables/reisezweck_keywords_general.html')

In [145]:
reisezweck_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='R_zweck', text_column='text_preprocessed_tokenized')
reisezweck_keywords = trending_keywords_to_dataframe(reisezweck_keywords)
#reisezweck_keywords
create_export_table(convert_resulttable_to_wide_format(reisezweck_keywords), filename='../../exports/tables/reisezweck_keywords_specific.html')

#### TF-IDF nach Billett

In [146]:
u_ticket_keywords_general = find_trending_keywords(df_text, filter_column='u_ticket', text_column='text_preprocessed_tokenized')
u_ticket_keywords_general = trending_keywords_to_dataframe(u_ticket_keywords_general)
#u_ticket_keywords
create_export_table(convert_resulttable_to_wide_format(u_ticket_keywords_general), filename='../../exports/tables/u_ticket_keywords_general.html')

In [147]:
u_ticket_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='u_ticket', text_column='text_preprocessed_tokenized')
u_ticket_keywords = trending_keywords_to_dataframe(u_ticket_keywords)
#u_ticket_keywords
create_export_table(convert_resulttable_to_wide_format(u_ticket_keywords), filename='../../exports/tables/u_ticket_keywords_specific.html')

#### TF-IDF nach Altersklasse

In [148]:
# fill missing values with -1
df_text['S_alter'] = pd.to_numeric(df_text['S_alter'], errors='coerce')
# define the age group bins
bins = [0, 24, 44, 64, 100]
# define the labels for the age groups
labels = ["1-24 Jahre", "25-44 Jahre", "45-64 Jahre", "65-100 Jahre"]

# use pd.cut() to create a new column with the age groups
df_text['S_alter_grouped'] = pd.cut(df_text['S_alter'], bins=bins, labels=labels)

In [149]:

S_alter_keywords_general = find_trending_keywords(df_text, filter_column='S_alter_grouped', text_column='text_preprocessed_tokenized')
S_alter_keywords_general = trending_keywords_to_dataframe(S_alter_keywords_general)
S_alter_keywords_general = S_alter_keywords_general.sort_values('Group') # Sort
# S_alter_keywords_general
create_export_table(convert_resulttable_to_wide_format(S_alter_keywords_general), filename='../../exports/tables/alter_keywords_group_general.html')

In [150]:
S_alter_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='S_alter_grouped', text_column='text_preprocessed_tokenized')
S_alter_keywords = trending_keywords_to_dataframe(S_alter_keywords)
S_alter_keywords = S_alter_keywords.sort_values('Group') # Sort
#season_keyword
create_export_table(convert_resulttable_to_wide_format(S_alter_keywords), filename='../../exports/tables/alter_keywords_group_specific.html')

#### TF-IDF nach Verkehrsmittel

In [151]:
ft_vm_kurz_keywords_general = find_trending_keywords(df_text, filter_column='ft_vm_kurz', text_column='text_preprocessed_tokenized')
ft_vm_kurz_keywords_general = trending_keywords_to_dataframe(ft_vm_kurz_keywords_general)
ft_vm_kurz_keywords_general = ft_vm_kurz_keywords_general.sort_values('Group') # Sort
#ft_vm_kurz_keywords
create_export_table(convert_resulttable_to_wide_format(ft_vm_kurz_keywords_general), filename='../../exports/tables/ft_vm_kurz_keywords_general.html')

In [152]:
ft_vm_kurz_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='ft_vm_kurz', text_column='text_preprocessed_tokenized')
ft_vm_kurz_keywords = trending_keywords_to_dataframe(ft_vm_kurz_keywords)
ft_vm_kurz_keywords = ft_vm_kurz_keywords.sort_values('Group') # Sort
#ft_vm_kurz_keywords
create_export_table(convert_resulttable_to_wide_format(ft_vm_kurz_keywords), filename='../../exports/tables/ft_vm_kurz_keywords_specific.html')

#### TF-IDF nach Transportunternehmen

In [153]:
ft_tu_keywords_general = find_trending_keywords(df_text, filter_column='ft_tu', text_column='text_preprocessed_tokenized')
ft_tu_keywords_general = trending_keywords_to_dataframe(ft_tu_keywords_general)
ft_tu_keywords_general = ft_tu_keywords_general.sort_values('Group') # Sort
#ft_tu_keywords
create_export_table(convert_resulttable_to_wide_format(ft_tu_keywords_general), filename='../../exports/tables/ft_tu_keywords_general.html')

In [154]:
ft_tu_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='ft_tu', text_column='text_preprocessed_tokenized')
ft_tu_keywords = trending_keywords_to_dataframe(ft_tu_keywords)
ft_tu_keywords = ft_tu_keywords.sort_values('Group') # Sort
#ft_tu_keywords
create_export_table(convert_resulttable_to_wide_format(ft_tu_keywords), filename='../../exports/tables/ft_tu_keywords_group_specific.html')

/Users/dom/Repos/github.com/DominikFin/nlp-satisfaction/code/functions.py:378: RuntimeWarning:

invalid value encountered in divide



#### TF-IDF nach Geschlecht

In [155]:
sex_keywords_general = find_trending_keywords(df_text, filter_column='S_sex', text_column='text_preprocessed_tokenized')
sex_keywords_general = trending_keywords_to_dataframe(sex_keywords_general)
#sex_keywords
create_export_table(convert_resulttable_to_wide_format(sex_keywords_general), filename='../../exports/tables/sex_keywords_group_general.html')

In [156]:
sex_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='S_sex', text_column='text_preprocessed_tokenized')
sex_keywords = trending_keywords_to_dataframe(sex_keywords)
#sex_keywords
create_export_table(convert_resulttable_to_wide_format(sex_keywords), filename='../../exports/tables/sex_keywords_group_specific.html')

#### TF-IDF nach Klasse

In [157]:
u_klassencode_keywords_general = find_trending_keywords(df_text, filter_column='u_klassencode', text_column='text_preprocessed_tokenized')
u_klassencode_keywords_general  = trending_keywords_to_dataframe(u_klassencode_keywords_general)
u_klassencode_keywords_general = u_klassencode_keywords_general.sort_values('Group') # Sort
#u_klassencode_keywords
create_export_table(convert_resulttable_to_wide_format(u_klassencode_keywords_general), filename='../../exports/tables/u_klassencode_keywords_general.html')

In [158]:
u_klassencode_keywords = find_trending_keywords_diff_normaized(df_text, filter_column='u_klassencode', text_column='text_preprocessed_tokenized')
u_klassencode_keywords  = trending_keywords_to_dataframe(u_klassencode_keywords )
u_klassencode_keywords = u_klassencode_keywords.sort_values('Group') # Sort
#u_klassencode_keywords
create_export_table(convert_resulttable_to_wide_format(u_klassencode_keywords), filename='../../exports/tables/u_klassencode_keywords_group_specific.html')

##### Export all Keywords in Excel File

In [159]:
# Select created dataframe of interest for export
dataframe_list = [
    year_keywords_general,
    year_keywords,
    yearquarter_keywords_general,
    yearquarter_keywords,
    yearseason_keywords_general,
    yearseason_keywords,
    season_keywords_general,
    season_keywords,
    yearmonth_keywords_general,
    yearmonth_keywords,
    u_ticket_keywords_general,
    u_ticket_keywords,
    S_alter_keywords_general,
    S_alter_keywords,
    sex_keywords_general,
    sex_keywords,
    u_klassencode_keywords_general,
    u_klassencode_keywords,
    ft_vm_kurz_keywords_general,
    ft_vm_kurz_keywords,
    ft_tu_keywords_general,
    ft_tu_keywords,
    ft_zielort_keywords_general,
    ft_zielort_keywords,
]

# Naming the df before export to have better sheetnames
year_keywords_general.name = 'year_keywords_general'
year_keywords.name = 'year_keywords_specific'
yearquarter_keywords_general.name = 'yearquarter_keywords_general'
yearquarter_keywords.name = 'yearquarter_keywords_specific'
yearseason_keywords_general.name = 'yearseason_keywords_general'
yearseason_keywords.name = 'yearseason_keywords_specific'
season_keywords_general.name = 'season_keywords_general'
season_keywords.name = 'season_keywords_specific'
yearmonth_keywords_general.name = 'yearmonth_keywords_general'
yearmonth_keywords.name = 'yearmonth_keywords_specific'
u_ticket_keywords_general.name = 'u_ticket_keywords_general'
u_ticket_keywords.name = 'u_ticket_keywords_specific'
S_alter_keywords_general.name = 'S_alter_keywords_general'
S_alter_keywords.name = 'S_alter_keywords_specific'
sex_keywords_general.name = 'sex_keywords_general'
sex_keywords.name = 'sex_keywords_specific'
u_klassencode_keywords_general.name = 'u_klassencode_keywords_general'
u_klassencode_keywords.name = 'u_klassencode_keywords_specific'
ft_vm_kurz_keywords_general.name = 'ft_vm_kurz_keywords_general'
ft_vm_kurz_keywords.name = 'ft_vm_kurz_keywords_specific'
ft_tu_keywords_general.name = 'ft_tu_keywords_general'
ft_tu_keywords.name = 'ft_tu_keywords_specific'
ft_zielort_keywords_general.name = 'ft_zielort_keywords_general'
ft_zielort_keywords.name = 'ft_zielort_keywords_specific'


# Export to results to slsx
def export_dataframes_to_excel(dataframe_list, file_path):
    # create a Pandas Excel writer using the file path and engine of your choice
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

    # loop through the data frames, create a new sheet for each one and write it to that sheet
    for i, df in enumerate(dataframe_list):
        sheet_name = df.name if hasattr(df, 'name') else f"Sheet{i+1}"
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # save the Excel file
    writer.save()

export_dataframes_to_excel(dataframe_list , '../../exports/tables/tfid_keywords.xlsx')

### Satisfaction Data

In [160]:
satisfaction_codes = config[config['vartype'] == 'satisfaction']['fragecode']

In [161]:
# # Pandas Profiling Satisfaction Data
# profile = ProfileReport(
#     df[
#         satisfaction_codes
#     ],
#     title="SatisfactionData",
#     lazy=False,
#     dark_mode=True,
# )

# profile.to_file("html/ProfilingSatisfactionData.html")


In [162]:
df_satisfaction = pd.melt(df, id_vars='yearmonth', value_vars=satisfaction_codes) #wide to long and only keep satisfaction questions
df_satisfaction = df_satisfaction.groupby(['yearmonth', 'variable'])[['value']].mean().reset_index() #groupy monthyear and variable

df_satisfaction_average = df_satisfaction.groupby(['yearmonth'])[['value']].mean().reset_index() #groupy monthyear to get overall average
df_satisfaction_average.insert(1,'variable',"Durchschnitt")

df_satisfaction = pd.concat([df_satisfaction,df_satisfaction_average],ignore_index=True).sort_values('yearmonth')

In [163]:
df_satisfaction

,yearmonth,variable,value
0,2019-01-01,wime_fahrplan,80.814030
8,2019-01-01,wime_sauberkeit,78.113531
7,2019-01-01,wime_puenktlich,89.157772
6,2019-01-01,wime_preis_leistung,72.375195
5,2019-01-01,wime_platzangebot,79.512920
...,...,...,...
411,2022-12-01,wime_preis_leistung,73.053322
412,2022-12-01,wime_puenktlich,88.130144
413,2022-12-01,wime_sauberkeit,77.366521
406,2022-12-01,wime_gesamtzuf,85.174942


In [164]:
def create_figure(df_satisfaction):
    fig = px.line(
        df_satisfaction,
        x="yearmonth",
        y="value",
        color="variable",
        color_discrete_sequence=color_discrete_sequence,
        template=template,
    )

    fig.update_layout(
        title="Zufriedenheit mit verschiedenen Aspekten im zeitlichen Verlauf",
        yaxis_title="Zufriedenheit",
        xaxis_title="",
        width=900, 
        height=450,
        legend_title="",
    )

    fig.update_traces(line_width=2)
    fig.update_yaxes(range=(62, 100))
    fig.add_vrect(
        x0="2020-03-25",
        x1="2022-03-01",
        line_width=0,
        fillcolor="#0B1F26",
        opacity=0.1,
        annotation_text="COVID-19 Massnahmen",
        annotation_position="top right",
        annotation_font_size=10,
        annotation_font_color="#0B1F26",
    )
    legend_labels = {
        'wime_fahrplan': 'Fahrplanangebot',
        'wime_sauberkeit': 'Sauberkeit',
        'wime_puenktlich': 'Pünktlichkeit',
        'wime_preis_leistung':'Preis Leistung',
        'wime_platzangebot':'Platzangebot',
        'wime_oes_fahrt':'Sicherheitsempfinden',
        'wime_komfort':'Ausstattung des Zuges',
        'wime_gesamtzuf':'Spontane Gesamtzufriedenheit',
        'wime_personal': 'Kundenbegleiter'}
        
    # sort the legend_labels dict by the values
    sorted_labels = dict(sorted(legend_labels.items(), key=lambda x: x[1]))
    for legend_group, label in sorted_labels.items():
        fig.update_traces(name=label, selector={'legendgroup': legend_group})
        
    line_width = 2
    fig.update_traces(patch={"line": {"dash": 'dot'}}, selector={"legendgroup": "Durchschnitt"})
    fig.show()
    pio.write_image(fig, "../../exports/images/fig-satisfaction_zeitreihe.svg")

create_figure(df_satisfaction=df_satisfaction) 


### MetaData

In [165]:
meta_codes = config[config['vartype'] == 'meta']['fragecode']

In [166]:
## Pandas Profiling MetaData
# profile = ProfileReport(
#     df[
#         meta_codes
#     ],
#     title="MetaData",
#     lazy=False,
#     dark_mode=True,
# )

# profile.to_file("html/ProfilingMetaData.html")
